In [1]:
import hyperspy.api as hs
from matplotlib import pyplot as plt
import numpy as np
import pycrystem as pc
plt.rcParams['image.cmap'] = 'gray'
%matplotlib tk

In [11]:
%%time
dp = pc.load("/home/pc494/Documents/data/GaN_intro/GaN_indent.blo")

CPU times: user 0 ns, sys: 860 ms, total: 860 ms
Wall time: 857 ms


In [12]:
dp = dp.inav[0:25,475:500] ## small to start

In [13]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]));

A Jupyter Widget

In [14]:
peaks = dp.find_peaks('max')
## stat is weird and slow

A Jupyter Widget

A Jupyter Widget

In [15]:
print(dp)
print(peaks)

<ElectronDiffraction, title: , dimensions: (25, 25|144, 144)>
<DiffractionVectors, title: , dimensions: (25, 25|)>


In [16]:
if dp.axes_manager.signal_shape[0] == dp.axes_manager.signal_shape[1]:
    peaks.data = peaks.data + (dp.axes_manager.signal_shape[0])/2 * 0.0076034063260340635
else:
    raise NotImplementedError('Needs a square signal')

In [8]:
def find_max_length_peaks(peaks):
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    length_of_longest_peaks_list = 0
    for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                if peaks.data[x,y].shape[0] > length_of_longest_peaks_list:
                    length_of_longest_peaks_list = peaks.data[x,y].shape[0]
    return length_of_longest_peaks_list      

In [17]:
## This is slow and needs to be vectorised
### Annoyingly I couldn't get it profiled

def generate_markers_from_peaks(peaks):
    markx,marky = [],[]
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    max_peak_count = find_max_length_peaks(peaks)
    for k in np.arange(0,max_peak_count):
        mx,my = [],[]
        for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                try:
                    mx.append(peaks.inav[x,y].data[k][1])
                    my.append(peaks.inav[x,y].data[k][0])
                except IndexError: #we have fewer than 'max_peak_count' peaks, so no marker
                    mx.append(np.nan)
                    my.append(np.nan)
        markx.append(np.asarray(mx).reshape(x_size,y_size).T)
        marky.append(np.asarray(my).reshape(x_size,y_size).T)
    
    return markx,marky 

In [146]:
def generate_markers_from_peaks_vect(peaks):
    max_peak_count = find_max_length_peaks(peaks)
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    markx,marky = [],[]
    for k in np.arange(0,max_peak_count):
        mx,my = np.empty([x_size,y_size]),np.empty([x_size,y_size]) #to be consistent with unvect
        for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                try:
                    mx[x,y] = (peaks.inav[x,y].data[k][1])
                    my[x,y] = (peaks.inav[x,y].data[k][0])
                except IndexError: #we have fewer than 'max_peak_count' peaks, so no marker
                    mx[x,y] = (np.nan)
                    my[x,y] = (np.nan)
                
        markx.append(mx.T)
        marky.append(my.T)
    
    return markx,marky 

In [147]:
%%time
markx,marky = generate_markers_from_peaks(peaks) #quarter second per 144x144 pattern

CPU times: user 14.9 s, sys: 20 ms, total: 14.9 s
Wall time: 14.8 s


In [19]:
%lprun?

In [105]:
%%time
#markxv,markyv = generate_markers_from_peaks_vect(peaks)

CPU times: user 2.62 s, sys: 20 ms, total: 2.64 s
Wall time: 2.54 s


We can try and do this in a more hyperspy-like way though